# Random Forest

__Random Forest__ 을 설명하기 앞서  
__Boostrapping__ 에 대해 설명드리겠습니다.  

> __Boostrapping__  
__Boostrapping__ 이란,  
훈련 데이터를 여러개로 나눈 후  
기존 데이터의 개수만큼 데이터 셋을 복원 추출하는 것을 말합니다.  

__Random Forest__ 의 경우 __Boostrapping__ 을 이용하여 기본 모델을 생성하며,  
__Bagging__ 보다 각 모델들의 분산을 줄여 학습하는 기법입니다.  


또한, __Tree__ 모델을 사용하여  

데이터 분포에 대한 가정 없어도 되며  
데이터 크기가 커도 빠르게 모델을 구축할 수 있습니다.



## 모델 생성 방식
1. 데이터를 여러 부분으로 나눕니다.  
2. 원 데이터의 크기 만큼 복원 추출합니다.  
3. 각 샘플에서 변수를 무작위 추출해 의사결정나무를 만듭니다.  
4. 각 모형들의 평균을 이용하여 값을 예측합니다.  

<img src="./image/Random Forest/Randomforest.gif" width="450" height="450">

## Tree의 구조

![structure](./image/Random Forest/structure.png)

__노드__ : 트리를 구성하는 기본 요소로, 위 그림에서 동그라미.  
__간선__ : 노드를 이어주는 선분  
__루트 노드__ : 데이터의 분할이 시작되는 최상위 노드, A 노드.  
__부모 노드__ : 하위의 노드가 있는 노드    
__자식 노드__ : 상위 노드가 있는 노드  
__형제 노드__ : 같은 상위 노드를 가진 노드들  
__리프 노드__ : 맨 끝 노드  
__깊이__ : 루트 노드로 부터 특정 노드 까지의 길이  
__높이__ : 루트 노드로 부터 가장 끝 노드 까지의 길이   

## Hyper Parameter

파라미터|설명|비고|초기값
---|---|---|---
__n_estimators__|의사결정나무의 개수 지정|개수를 늘리면 성능이 좋아지나 시간도 많이 걸림|default = 10
__min_samples_split__|노드를 분할하기 위한 최소한의 샘플 데이터 수|과적합 제어에 사용|default = 2
__min_saples_leaf__|리프 노드가 되기 위해 필요한 최소한의 샘플 데이터 수|과적합 제어에 사용|default = 1
__max_feature__|나무를 분할하는데 사용할 변수의 개수|일반적으로 변수의 개수를 루트로 사용|default = 'auto' (=squrt)
__max_depth__|의사결정나무의 깊이 설정|깊이가 길어지면 과적합 위험|default = None
__oob_score__|OOB(out of bag) 설정|error율 판단 척도|default = False


## 코드

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier # 분류 트리
from sklearn.ensemble import RandomForestRegressor # 회귀 트리
from sklearn.model_selection import train_test_split # train/test set 나누기
from sklearn.metrics import accuracy_score # 정확도 측정

In [0]:
data = pd.read_csv('./train.csv')

features = list(data.columns.difference('target'))
X = data[features] # 설명 변수
y =  data['target'] # 반응 변수

X_train, X_test, y_train, y_test =\
        train_test_split(X, y, test_size = 0.2, random_state = 2020) # 데이터 * 0.8 = train set / 데이터 * 0.2 = vaild set

## 모델 생성

In [0]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print('Accuracy : %.2f' %(accuracy_score(y_test, pred) * 100), "%")

In [0]:
model2 = RandomForestClassifier(n_estimators = 200,
                                max_depth = 10,
                                random_state = 2020,
                                oob_score = True)
model2.fit(X_train, y_train)
pred2 = model2.predict(X_test)
print('Accuracy : %.2f' %(accuracy_score(y_test, pred2) * 100), "%")

## 변수 중요도 시각화

In [0]:
imp = model.feature_importances_
feature_imp = pd.Series(imp, index = X_train.columns)
top_10 = feature_imp.sort_values(ascending = False)[:10]

plt.figure(figsize = (18, 8))
plt.title('Feature importances')
sns.barplot(x = top_10, y = top_10.index)

## Grid search

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
n_estimators = [10, 20, 40, 100]
max_features = [2, 4]
max_depth = [10]
random_state = [2020]

param_grid = [
    {'n_estimators' : n_estimators, 'max_features' : max_features, 'random_state' : random_state} ,
    {'max_depth' : max_depth, 'n_estimators' : n_estimators, 'max_features' : max_features, 'random_state' : random_state}
]

model = RandomForestClassifier()
grid_search = GridSearchCV(model,
                           param_grid = param_grid,
                           cv = 2,
                           scoring = 'accuracy',
                           verbose = 1,
                           n_jobs = 1)

grid_search.fit(X_train, y_train)

In [0]:
grid_search.best_params_

In [0]:
grid_search.best_estimator_

## Random Search

In [0]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(i) for i in range(10, 100, 10)]
max_features = ['log2', 'auto']
max_depth = [int(i) for i in np.linspace(start = 10, stop = 110, num = 11)] # 10에서 110까지 동일한 간격으로 11개 만듦
min_sample_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
random_state = [2020]

random_grid = {
    'n_estimators' : n_estimators,
    'max_features' : max_features,
    'max_depth' : max_depth,
    'min_sample_split' : min_sample_split,
    'min_samples_leaf' : min_samples_leaf,
    'random_state' : random_state
}

model = RandomForestClassifier()
random_search = RandomizedSearchCV(model,
                                   param_distributions = random_grid,
                                   cv = 2,
                                   n_iter = 10,
                                   scoring = 'accuracy',
                                   verbose = 2, 
                                   n_jobs = 4)
random_search.fit(X_train, y_train)

## Reference

### Tree
- [[자료구조/java] 트리 (Tree)](https://songeunjung92.tistory.com/m/26)

### Random Forest
- [랜덤 포레스트](https://www.wikiwand.com/ko/%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8)  
- [[Chapter 4. 분류] 랜덤포레스트(Random Forest)](https://injo.tistory.com/30)  
- [<MACHINE LEARNING>하이퍼파라미터 튜닝](https://databuzz-team.github.io/2018/12/05/hyperparameter-setting/)
- [Hyperparameter Tuning the Random Forest in Python](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)